In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import csv

#To select the test-train set, metrics etc.
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
#from imblearn.over_sampling import SMOTE

#Our models
from sklearn.ensemble import RandomForestClassifier #
#from xgboost import XGBClassifier 
#from lightgbm import LGBMClassifier
from sklearn import tree
#from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression #
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
hr = pd.read_csv(r"E:\PycharmProjects\Inputs\Sales\Sales_Data1.csv")

In [ ]:
hr.head()

In [ ]:
print('UNIQUE Material Number:  ' + hr.MaterialNumber.unique())

print('UNIQUE Channel:  ' + hr.DistChannel_Name.unique())

print('UNIQUE Division:  ' + str(hr.Division.unique()))

print('UNIQUE MSO:  ' + str(hr.MSO.unique()))

#print('UNIQUE DivisionDesc:  ' + hr.DivisionDesc.unique())

In [ ]:
havells = ['CONSUMER LIGHTING','HAVELLS WIRE', 'HAVELLS EWA, HAVELLS BCP', 'HAVELLS FAN', 'HAVELLS SDA',
         'HAVELLS WATER HEATER, HAVELLS IP', 'HAVELLS PUMP, HAVELLS CAPACITOR', 'HAVELLS AIR COOLER']
lloyd = [ 'LLOYD WASHING MACHIN, LLOYD REFRIGERATORS', 'LLOYD AIR CONDITIONE, LLOYD LED TELEVISION',
 'LLOYD OTHERS']

others = ['REO EWA', 'CRABTREE EWA', 'REO BCP',
           'REO WIRE', 'MFD CAPACITORS']

standard = ['STANDARD CP', 'STANDARD WATERHEATER', 'STANDARD FAN', 'STANDARD WIRE']

def apply(job):
    if job.strip(' ') in havells: return 'havells'
    if job.strip(' ') in lloyd: return 'lloyd'
    if job.strip(' ') in others: return 'others'
    if job.strip(' ') in standard: return 'standard'
    
for i in range(0,hr.shape[0]):
    hr.at[i,'DivisionDesc'] = apply(hr.iloc[i]['DivisionDesc'])

In [ ]:
hr.head()

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(hr.DistChannel_Name.unique())
hr.DistChannel_Name=le.transform(hr.DistChannel_Name)

#le.fit(hr.DivisionDesc.unique())
#hr.DivisionDesc=le.transform(hr.DivisionDesc)

le.fit(hr.MaterialNumber.unique())
hr.MaterialNumber=le.transform(hr.MaterialNumber)

hr = hr.drop('BillingDocument', axis=1)

hr.head()

#hr.to_csv('E:/PycharmProjects/Inputs/Sales/transformed.csv', index=False)

In [ ]:

rows = []
city_pop = {}
detail =[1]
with open('E:/PycharmProjects/Inputs/Sales/population.csv', 'r') as f:
    line = csv.reader(f)
    for row in line:
        rows.append(row)
f.close()

for i in rows:
    detail = i[0].split('\t')
    city = detail[0].upper()
    city_pop.fromkeys(city)
    city_pop[city] = detail[1]
    
# for k, v in city_pop.items():
#     print(f'{k}-{v}')
    
#print(city_pop)

def city(pop):
    print(pop)
    try:
        str = 'rural'
        if (int(pop)>=100000) & (int(pop)<3000000): str = 'town'
        if (int(pop)>=3000000) & (int(pop)<9000000): str = 'city'
        if int(pop)>=9000000: str = 'mega'
        
    except:
        str = 'err'
    print(str)
    return str

temp = hr['MSOName'].map(city_pop, na_action='error')

for i in range(0,hr.shape[0]):
    hr.at[i,'MSOName'] = city(temp[i])

In [ ]:
hr.head(500)
hr.to_csv('E:/PycharmProjects/Inputs/Sales/transformed_pop.csv', index=False)

In [ ]:
term_perc = (hr_term.shape[0]/(hr_term.shape[0]+hr_active.shape[0])*100)
act_perc = (hr_active.shape[0]/(hr_term.shape[0]+hr_active.shape[0])*100)

plt.bar(["Terminated", "Active"],[term_perc, act_perc],
        color = '#cf2f6b' )
plt.xlabel("Employees")
plt.ylabel("Percent")
plt.title("Working Status by %")
plt.ylim([0, 100])
plt.yticks([term_perc, act_perc], ['25%', '75%'])
plt.show()

In [ ]:
fig = plt.figure(figsize = (20, 10))

plt.subplot(221)
sns.set_style('white')
plt.title('age', size = 14)
sns.kdeplot(hr_term['age'], color = '#45258D', shade = True, label = 'terminated', alpha = 0.5)
sns.kdeplot(hr_active['age'], color = '#E00530', shade = True, label = 'active', alpha = 0.5)
plt.grid(color = 'gray', linestyle = ':', axis = 'x', zorder = 0,  dashes = (1,7))
plt.ylabel('')
plt.xlabel('')
plt.yticks([])
plt.legend(loc = 'upper left')

plt.subplot(222)
sns.set_style('white')
plt.title('length_of_service', size = 14)
sns.kdeplot(hr_term['length_of_service'], color = '#32384D', shade = True, label = 'terminated', alpha = 0.5)
sns.kdeplot(hr_active['length_of_service'], color = '#E29930', shade = True, label = 'active', alpha = 0.5)
plt.grid(color = 'gray', linestyle = ':', axis = 'x', zorder = 0,  dashes = (1,7))
plt.ylabel('')
plt.xlabel('')
plt.yticks([])
plt.legend(loc = 'upper left')

plt.subplot(223)
sns.set_style('white')
plt.title('STATUS_YEAR', size = 14)
sns.distplot(hr_term,x=hr_term['STATUS_YEAR'], color = '#32384D', label = 'terminated')
plt.grid(color = 'grey', linestyle = ':', axis = 'x', zorder = 0,  dashes = (1,7))
plt.ylabel('')
plt.xlabel('')
plt.yticks([])
plt.legend(loc = 'upper left')


plt.subplot(224)
sns.set_style('white')
plt.title('store_name', size = 14)
sns.distplot(hr_term,x=hr_term['store_name'], color = '#32384D', label = 'terminated', kde=False)
sns.distplot(hr_active,x=hr_active['store_name'], color = '#E29930', label = 'terminated', kde=False)
plt.grid(color = 'gray', linestyle = ':', axis = 'x', zorder = 0,  dashes = (1,7))
plt.ylabel('')
plt.xlabel('')
plt.yticks([])
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(hr.city_name.unique())
hr.city_name=le.transform(hr.city_name)
hr_active.city_name = le.transform(hr_active.city_name)
hr_term.city_name=le.transform(hr_term.city_name)


le.fit(hr.job_title.unique())
hr.job_title=le.transform(hr.job_title)
hr_active.job_title = le.transform(hr_active.job_title)
hr_term.job_title=le.transform(hr_term.job_title)



le.fit(hr.termreason_desc.unique())
hr.termreason_desc=le.transform(hr.termreason_desc)
hr_active.termreason_desc = le.transform(hr_active.termreason_desc)
hr_term.termreason_desc=le.transform(hr_term.termreason_desc)


le.fit(hr.termtype_desc.unique())
hr.termtype_desc=le.transform(hr.termtype_desc)
hr_active.termtype_desc = le.transform(hr_active.termtype_desc)
hr_term.termtype_desc=le.transform(hr_term.termtype_desc)



le.fit(hr.STATUS.unique())
hr.STATUS=le.transform(hr.STATUS)
hr_active.STATUS = le.transform(hr_active.STATUS)
hr_term.STATUS=le.transform(hr_term.STATUS)


le.fit(hr.BUSINESS_UNIT.unique())
hr.BUSINESS_UNIT=le.transform(hr.BUSINESS_UNIT)
hr_active.BUSINESS_UNIT = le.transform(hr_active.BUSINESS_UNIT)
hr_term.BUSINESS_UNIT=le.transform(hr_term.BUSINESS_UNIT)



le.fit(hr.department_name.unique())
hr.department_name=le.transform(hr.department_name)
hr_active.department_name = le.transform(hr_active.department_name)
hr_term.department_name=le.transform(hr_term.department_name)


le.fit(hr.gender_short.unique())
hr.gender_short=le.transform(hr.gender_short)
hr_active.gender_short = le.transform(hr_active.gender_short)
hr_term.gender_short=le.transform(hr_term.gender_short)


In [ ]:
hr_final=pd.concat([hr_active, hr_term]).reset_index()
hr_corr=hr_final.corr()
plt.figure(figsize=(13, 11))
sns.heatmap(hr_corr, annot = True, cmap = 'coolwarm', fmt=".2f",
            vmin = -1, vmax = 1, linewidths = 0.15, linecolor = 'white', cbar = True)
plt.show()

In [ ]:
hr_corr.abs()['STATUS'].sort_values(ascending=False)

In [ ]:
hr_final=hr_final.drop(['termreason_desc', 'termtype_desc', 'index', 'orighiredate_key', 'terminationdate_key'], axis=1)

y=hr_final.STATUS
hr_final=hr_final.drop('STATUS', axis=1)
hr_final.head() 


In [ ]:
hr_final.to_csv(r"C:\Users\33669\PycharmProjects\Inputs\processedout.csv", index = False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(hr_final, y, test_size = 0.2, random_state = 2022)

#smote = SMOTE()
#X_train, y_train = smote.fit_resample(X_train, y_train)
#y_train.value_counts()

In [ ]:
import pickle
results = pd.DataFrame(columns = ['LR', "SVC"], index = range(4))
def models(model, X_train, X_test, y_train, y_test, i):
    model.fit(X_train, y_train)
    modelname = 'svm'
    with open('svm.pkl', 'wb') as f:
        pickle.dump(model, f)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:,1]
    
 
    frame = {'Service Time' : X_test.length_of_service, 'Actual Outcome' : y_test, 'Predicted Outcome' : y_pred,
             'Likeliness/Confidence' : y_prob}
    df = pd.DataFrame(frame)
    df.head()
    df.to_csv(r"C:\Users\33669\PycharmProjects\Inputs\output.csv", index = False)
    
    # Metrics
    model_cm = confusion_matrix(y_test, y_pred)
    results.iloc[0, i] = round(precision_score(y_test, y_pred), 2)
    results.iloc[1, i] = round(recall_score(y_test, y_pred), 2)
    results.iloc[2, i] = round(f1_score(y_test, y_pred), 2)
    results.iloc[3, i] = round(roc_auc_score(y_test, y_prob), 3)
    print(round(precision_score(y_test, y_pred), 2))
    print(round(recall_score(y_test, y_pred), 2))
    print(round(f1_score(y_test, y_pred), 2))
    print(round(roc_auc_score(y_test, y_prob), 3))

    print(classification_report(y_test, y_pred))
    print(f'ROC AUC score: {round(roc_auc_score(y_test, y_prob), 3)}')
    print('')
    print('-----------------------------------------------------')
    print('')
    print('Cross-validation scores with 5 folds:')
    print('')
    print(f"ROC AUC: {round(cross_val_score(model, X_train, y_train, cv = 5, scoring = 'roc_auc').mean(), 3)}")
    print(f"precision: {round(cross_val_score(model, X_train, y_train, cv = 5, scoring = 'precision').mean(), 3)}")
    print(f"recall: {round(cross_val_score(model, X_train, y_train, cv = 5, scoring = 'recall').mean(), 3)}")
    print(f"f1: {round(cross_val_score(model, X_train, y_train, cv = 5, scoring = 'f1').mean(), 3)}")

    # Visualize confusion matrix
    plt.figure(figsize = (14, 5))
    plt.subplot(121)
    sns.heatmap(model_cm, cmap = 'YlOrBr', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15},
               yticklabels = ['Not looking for job change', 'Looking for job change'], xticklabels = ['Predicted not looking for job change', 'Predicted looking for job change'])
    plt.yticks(rotation = 0)

    # Roc curve
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    
    plt.subplot(122)
    sns.set_theme(style = 'white')
    plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
    plt.axis('tight')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

#svc = SVC(kernel='linear', C = 1.0, probability = True)
#models = (svc, X_train, X_test, y_train, y_test, 0)
svm=SVC(kernel='linear', C = 1.0, probability = True)
models(svm, X_train, X_test, y_train, y_test, 0)